In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
import json
from os import listdir
import glob
from scipy import spatial


import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
import spacy

[nltk_data] Downloading package stopwords to C:\Users\S
[nltk_data]     RITIKA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

import warnings
warnings.filterwarnings('ignore')

In [4]:
def cosine_similarity(arr1,arr2):
    ans=1- spatial.distance.cosine(arr1,arr2)
    if(np.isnan(ans)):
        return 0
    else:
        return ans

In [5]:
nlp=spacy.load('en_core_web_lg')

In [6]:
class job_postings:    
    def __init__(self,link):
        self.df2=pd.read_csv(link)
        self.training_range=int(len(self.df2.loc[:,'uniq_id']))
        
    

    def categorize_jobs(self):
        def check_threshold(threshold,ele):
            if(ele[0]!=threshold[0][0] and abs(ele[1]-threshold[0][1])<0.03):
                return True
            else:
                return False
       
        #Compare similarities of word embeddings
        
        job_id=self.df2.loc[:,'uniq_id'].tolist()[:self.training_range]
        job_titles=self.df2.loc[:,'jobtitle'].tolist()[:self.training_range]
        job_descriptions=self.df2.loc[:,'jobdescription'].tolist()[:self.training_range]
        final_cat=pd.DataFrame(index=job_id)
        
        categories=['Network Engineer','Full stack','QA/Test Developer','Enterprise application','DevOps',
                    'Mobile Developer','Back End','Database Administrator(DBA)','Front End','Game developer',
                    'System Administrator','Data Scientist','Business analyst','Sales professional',
                    'Product Manager','Information Security','Software Developer/Java Developer',
                    'Web Developer','Cloud Computing']
        for category in categories:
            final_cat[category]=np.nan
        for job_t_d in list(zip(job_id,job_titles,job_descriptions)):
            id_job=job_t_d[0]
            job_i=job_t_d[1]
            job_d=job_t_d[2]
            job_title=nlp(job_i.lower())
            job_description=nlp(job_d.lower())
            match_cat_title=dict()
            match_cat_description=dict()
            for category in categories:
                word=nlp(category.lower())
                match_cat_title[category]=job_title.similarity(word)
                match_cat_description[category]=job_description.similarity(word)
            match_cat_title=sorted(match_cat_title.items(),key=lambda x:x[1],reverse=True)
            match_cat_description=sorted(match_cat_description.items(),key=lambda x:x[1],reverse=True)
        
            a=match_cat_title[0]
            #print(a)
            match_cat_description=list(filter(lambda x:check_threshold(match_cat_title,x),match_cat_description))
            if(len(match_cat_description)!=0):
                print(match_cat_description)
                print(id_job)
               
                final_cat.loc[id_job,a[0]]=1
                match_cat_description.extend([(match_cat_title[0][0],1)])
                sum_proportion=sum([x[1] for x in match_cat_description])
                for ele in match_cat_description:
                    final_cat.loc[id_job,ele[0]]=ele[1]/sum_proportion
            else:
                print(id_job)
                final_cat.loc[id_job,a[0]]=1
        return final_cat

    def clean_skills(self):
        extracted_skills=dict()
        job_skills=np.asarray(self.df2.loc[:,"skills"])
        for i in range(self.training_range):
            
            job_id=self.df2.iloc[i,-1]
            
            tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
            #print(job_skills[i])
            if(pd.isnull(job_skills[i])):
                continue
            stopwords_list=stopwords.words("english")
            tokens=re.split("|".join([","," and","/"," AND"," or"," OR",";"]),job_skills[i])
            tokens=list(set(tokens))
            extracted_skills[job_id]=[]
            extracted_skills[job_id].extend(tokens)
        return extracted_skills

    def extract_skills(self,extracted_skills):
        df_languages=pd.read_excel('./data/languages.xlsx')
        df_frameworks=pd.read_csv("./data/frameworks.csv")
        df_database=pd.read_csv("./data/database.csv")
        df_os=pd.read_csv("./data/operating_systems.csv")
        df_plat=pd.read_csv("./data/platforms.csv")
        frameworks=df_frameworks.iloc[:,1].tolist()
        frameworks=[x.lower().strip() for x in frameworks]
        languages=list(df_languages.iloc[:,0])
        languages=[x.lower().strip() for x in languages]
        databases=df_database.iloc[:,0].tolist()
        databases=[x.lower().strip() for x in databases]
        op_systems=df_os.iloc[:,0].tolist()
        op_systems=[x.lower().strip() for x in op_systems]
        platforms=df_plat.iloc[:,1].tolist()
        
        platforms=[x.lower().strip() for x in platforms]
        new_extracted=dict()
        for ele in extracted_skills.keys():
            final_lang=''
            final_frame=''
            final_others=''
            final_database=''
            final_plat=''
            final_os=''
            final_other=''
            for skill in extracted_skills[ele]:
                skill_base=skill.lower().strip()
                
                if(skill_base in languages):
                    if(final_lang==''):
                        final_lang=skill_base
                    else:
                        final_lang=final_lang+","+skill_base
                elif(skill_base in frameworks):
                    if(final_frame==''):
                        final_frame=skill_base
                    else:
                        final_frame=final_frame+","+skill_base
                elif(skill_base in databases):
                    if(final_database==''):
                        final_database=skill_base
                    else:
                        final_database=final_database+","+skill_base
                elif(skill_base in op_systems):
                    if(final_os==''):
                        final_os=skill_base
                    else:
                        final_os=final_os+","+skill_base
                elif(skill_base in platforms):
                    if(final_plat==''):
                        final_plat=skill_base
                    else:
                        final_plat=final_plat+","+skill_base
                else:
                    if(skill_base!='null'or skill_base!='none'):
                        if(final_others==''):
                            final_others=skill_base
                        else:
                            final_others=final_others+","+skill_base
            new_extracted[ele]=[final_lang,final_frame,final_database,final_os,final_plat,final_others]
        print((list(new_extracted.items()))[:100])
        for ele,describe in list(zip(self.df2.loc[:,'uniq_id'],self.df2.loc[:,'jobdescription'].tolist()))[:self.training_range]:
            doc=nlp(describe)
            final_lang=''
            final_frame=''
            final_others=''
            final_database=''
            final_plat=''
            final_os=''
            for ent in doc.ents:
                word=ent.text
                word=word.lower().strip()
                try:
                    if(word in languages and word not in final_lang and word not in new_extracted[ele][0].split(",")):
                        if(final_lang==''):
                            final_lang=word
                        else:
                            final_lang=final_lang+","+word
                    elif(word in frameworks and word not in final_frame and word not in new_extracted[ele][1].split(",")):
                        if(final_frame==''):
                            final_frame=word
                        else:
                            final_frame=final_frame+","+word
                    elif(word in databases and word not in final_database and word not in new_extracted[ele][2].split(",")):
                        if(final_database==''):
                            final_database=word
                        else:
                             final_database=final_database+","+word
                    elif(word in op_systems and word not in final_os and word not in new_extracted[ele][3].split(",")):
                        if(final_os==''):
                            final_os=word
                        else:
                            final_os=final_os+","+word
                    elif(word in platforms and word not in final_plat and word not in new_extracted[ele][4].split(",")):
                        if(final_plat==''):
                            final_plat=word
                        else:
                            final_plat=final_plat+","+word
                    else:
                        if(final_others==''):
                            final_others=word
                        else:
                            final_others=final_others+","+word
                except KeyError:
                    pass
                
            
            if(final_lang!=''):
                new_extracted[ele][0]+=","+final_lang
            if(final_frame!=''):
                new_extracted[ele][1]+=","+final_frame
            if(final_database!=''):
                new_extracted[ele][2]+=","+final_database
            if(final_os!=''):
                new_extracted[ele][3]+=","+final_os
            if(final_plat!=''):
                new_extracted[ele][4]+=","+final_plat
           # if(final_others!=''):
            #    new_extracted[ele][5]+=","+final_others
           
           
        extracted_skills_df=pd.DataFrame.from_dict(new_extracted,orient='index',columns=['Language','Framework','Database','OS','Platform','Others'])
        
        return extracted_skills_df    
            

    def create_job_profile(self,extracted_skills_df,domain_df):
        job_id=extracted_skills_df.index.tolist()
        languages_df=pd.DataFrame(index=job_id)
        platforms_df=pd.DataFrame(index=job_id)
        frameworks_df=pd.DataFrame(index=job_id)
        databases_df=pd.DataFrame(index=job_id)
        
        for job,lang,frame,plat,datab in list(zip(job_id,extracted_skills_df.loc[:,'Language'].tolist(),extracted_skills_df.loc[:,'Framework'].tolist(),extracted_skills_df.loc[:,'Platform'].tolist(),extracted_skills_df.loc[:,'Database'].tolist())):
            #Languages
            l=lang.split(",")
            if(lang!=np.nan or lang!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in languages_df.columns):
                        #languages.append(ele)
                        languages_df[ele]=np.nan
                    languages_df.loc[job,ele]=1
            
            #Frameworks
            l=frame.split(",")
            if(frame!=np.nan or frame!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in frameworks_df.columns):
                        #languages.append(ele)
                        frameworks_df[ele]=np.nan
                    frameworks_df.loc[job,ele]=1
            #Platforms
            l=plat.split(",")
            if(plat!=np.nan or plat!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in platforms_df.columns):
                        #languages.append(ele)
                        platforms_df[ele]=np.nan
                    platforms_df.loc[job,ele]=1
            
            #Databases
            l=datab.split(",")
            if(datab!=np.nan or datab!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in databases_df.columns):
                        #languages.append(ele)
                        databases_df[ele]=np.nan
                    databases_df.loc[job,ele]=1
        languages_df=languages_df.reindex(sorted(languages_df.columns), axis=1)
        frameworks_df=frameworks_df.reindex(sorted(frameworks_df.columns), axis=1)
        platforms_df=platforms_df.reindex(sorted(platforms_df.columns), axis=1)
        databases_df=databases_df.reindex(sorted(databases_df.columns), axis=1)
        domain_df=domain_df.reindex(sorted(domain_df.columns), axis=1)
        
        languages_df.index.name=frameworks_df.index.name=platforms_df.index.name=databases_df.index.name=domain_df.index.name='uniq_id'
        languages_df.to_csv("./data/job_profile/languages_jobprofile.csv")
        frameworks_df.to_csv("./data/job_profile/frameworks_jobprofile.csv")
        platforms_df.to_csv("./data/job_profile/platforms_jobprofile.csv")
        databases_df.to_csv("./data/job_profile/databases_jobprofile.csv")
        domain_df.to_csv("./data/job_profile/domain_jobprofile.csv")
        print(list(languages_df.columns))
        print(list(frameworks_df.columns))
        print(list(platforms_df.columns))
        print(list(databases_df.columns))
        print(list(domain_df.columns))
                
            
    
    def clean_common_profile(self,df_user,df_job,flag):
        #Shift .net from languages to frameworks
        if(flag=='Language'):
            print(df_job.columns.tolist())
            #bash and bash/shell
            count=0
            for ele in df_user.loc[:,'bash/shell']:
                if(ele==1.0):
                    df_user.loc[count,'bash']=1.0
                count=count+1
            df_user=df_user.drop('bash/shell',axis=1)
            count=0
            for ele in df_job.loc[:,'bash/shell']:
                if(ele==1.0):
                    df_job.loc[count,'bash']=1.0
                count=count+1
            df_job=df_job.drop('bash/shell',axis=1)

        if(flag=='Framework'):
            print(df_user.columns.tolist())
            count=0
            for ele in df_user.loc[:,'nodejs']:
                if(ele==1.0):
                    df_user.loc[count,'node.js']=1.0
                count=count+1
            df_user=df_user.drop('nodejs',axis=1)
            count=0
            for ele in df_job.loc[:,'nodejs']:
                if(ele==1.0):
                    df_job.loc[count,'node.js']=1.0
                count=count+1
            df_job=df_job.drop('nodejs',axis=1)
            
            count=0
            for ele in df_user.loc[:,'angularjs']:
                if(ele==1.0):
                    df_user.loc[count,'angular']=1.0
                count=count+1
            df_user=df_user.drop('angularjs',axis=1)
            count=0
            for ele in df_job.loc[:,'angularjs']:
                if(ele==1.0):
                    df_job.loc[count,'angular']=1.0
                count=count+1
            df_job=df_job.drop('angularjs',axis=1)
            
        if(flag=='Platform'):
            print(df_user.columns.tolist())
        if(flag=='Database'):
            print(df_user.columns.tolist())
            count=0
            for ele in df_user.loc[:,'microsoft sql server']:
                if(ele==1.0):
                    df_user.loc[count,'sql server']=1.0
                count=count+1
            df_user=df_user.drop('microsoft sql server',axis=1)
            count=0
            for ele in df_job.loc[:,'microsoft sql server']:
                if(ele==1.0):
                    df_job.loc[count,'sql server']=1.0
                count=count+1
            df_job=df_job.drop('microsoft sql server',axis=1)
        return df_user,df_job
    #Input is two dataframes    
    def create_common_profile(self,job_profile_path,user_profile_path,output_path,flag=0):
        if(flag==0):
            #Domain
            userprofile=pd.read_csv(user_profile_path+"DevType.csv",index_col='Respondent')
            jobprofile=pd.read_csv(job_profile_path+"domain_jobprofile.csv",index_col='Unnamed: 0')
            print("Read from file")
            print(jobprofile.index)
            #jobprofile=jobprofile.reset_index()
            #userprofile=userprofile.reset_index()
            userprofile.drop('Unnamed: 0', axis=1, inplace=True)
            jobprofile.drop('uniq_id', axis=1, inplace=True)
            jobprofile.index.name='uniq_id'
            print("index 2in domain")
            print(jobprofile.index)
            #print(jobprofile.loc[:,'uniq_id'])
            userprofile.rename(columns={'Product manager':'Product Manager','Back-end developer':'Back End','C-suite executive (CEO, CTO, etc.)':'C-suite executive','Data scientist or machine learning specialist':'Data Scientist','Database administrator':'Database Administrator(DBA)','Mobile developer':'Mobile Developer','Desktop or enterprise applications developer':'Enterprise application','DevOps specialist':'DevOps','Front-end developer':'Front End','Full-stack developer':'Full stack','Marketing or sales professional':'Sales professional','QA or test developer':'QA/Test Developer','System administrator':'System Administrator','Game or graphics developer':'Game developer'},inplace=True)
            jobprofile.rename(columns={'Business analyst':'Data or business analyst'},inplace=True)
            print(userprofile.columns)
            print(jobprofile.columns)
            print("index in domain")
            print(jobprofile.index)
            #Present in userprofile but not in jobprofile
            a=list(set(userprofile.columns)-set(jobprofile.columns))
            print(a)
            for i in a:
                if(i!='Respondent'):
                    jobprofile[i]=0
            b=list(set(jobprofile.columns)-set(userprofile.columns))
            print(b)
            for i in b:
                if(i!='uniq_id'):
                    userprofile[i]=0
            
            userprofile=userprofile[sorted(userprofile.columns.tolist())]
            jobprofile=jobprofile[sorted(jobprofile.columns.tolist())]
            #Exclude 

            print(userprofile.columns==jobprofile.columns)

            print(userprofile.columns)
            print(jobprofile.columns)
            userprofile=userprofile[userprofile.columns.tolist()]
            jobprofile=jobprofile[jobprofile.columns.tolist()]
            userprofile.to_csv(output_path+"domain_userprofile.csv")
            jobprofile.to_csv(output_path+"domain_jobprofile.csv")
            #Languages
            df_user=pd.read_csv(user_profile_path+"LanguageWorkedWith.csv",index_col='Respondent')
            df_job=pd.read_csv(job_profile_path+"languages_jobprofile.csv",index_col=0)
            df_job.index.name='uniq_id'
            print("index is")
            print(df_job.index)
            print(df_user.columns)
            print(df_job.columns)
            df_user.drop('Unnamed: 0', axis=1, inplace=True)
            #df_job.drop('Unnamed: 0', axis=1, inplace=True)
            df_job.rename(columns={'visual basic .net':'vb.net'},inplace=True)
            df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
            df_job.columns=list(map(lambda x:x.lower(),df_job.columns))
            columns_to_add=[]
            a=list(set(df_user.columns)-(set(df_job.columns)))
            print(a)
            for i in a:
                if(i!='Respondent'):
                    df_job[i]=0        
            b=list(set(df_job.columns)-set(df_user.columns))
            print(b)
            for i in b:
                if(i!='uniq_id'):
                    df_user[i]=0
            print(df_job.index)        
            df_user=df_user[sorted(df_user.columns.tolist())]
            df_job=df_job[sorted(df_job.columns.tolist())]
            print("index 2")
            print(df_job.index)
            print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
            df_user,df_job=self.clean_common_profile(df_user,df_job,'Language')
            print("language is")
            print(df_job.index[0])
            print(df_job.loc[df_job.index[0],:])
            df_user.to_csv(output_path+"languages_profileuser.csv")
            df_job.to_csv(output_path+"languages_profilejob.csv")
            #Frameworks
            df_user=pd.read_csv(user_profile_path+"FrameworkWorkedWith.csv",index_col='Respondent')
            df_job=pd.read_csv(job_profile_path+"frameworks_jobprofile.csv",index_col=0) 
            df_job.index.name='uniq_id'
            print(df_user.columns)
            print(df_job.columns)
            df_user.drop('Unnamed: 0', axis=1, inplace=True)
         
            df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
            df_job.columns=list(map(lambda x:x.lower(),df_job.columns))

            a=list(set(df_user.columns)-(set(df_job.columns)))
            print(a)
            for i in a:
                if(i!='Respondent'):
                    df_job[i]=0        
            b=list(set(df_job.columns)-set(df_user.columns))
            print(b)
            for i in b:
                if(i!='uniq_id'):
                    df_user[i]=0
            df_user=df_user[sorted(df_user.columns.tolist())]
            df_job=df_job[sorted(df_job.columns.tolist())]

            print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
            df_user,df_job=self.clean_common_profile(df_user,df_job,'Framework')   
            df_user.to_csv(output_path+"frameworks_profileuser.csv")
            df_job.to_csv(output_path+"frameworks_profilejob.csv")
            #Platforms
            df_user=pd.read_csv(user_profile_path+"PlatformWorkedWith.csv",index_col='Respondent')
            df_job=pd.read_csv(job_profile_path+"platforms_jobprofile.csv",index_col=0) 
            print(df_user.columns)
            df_job.index.name='uniq_id'
            print(df_job.columns)
            df_user.drop('Unnamed: 0', axis=1, inplace=True)
            df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
            df_job.columns=list(map(lambda x:x.lower(),df_job.columns))

            a=list(set(df_user.columns)-(set(df_job.columns)))
            print(a)
            for i in a:
                if(i!='Respondent'):
                    df_job[i]=0
            b=list(set(df_job.columns)-set(df_user.columns))
            print(b)
            for i in b:
                if(i!='uniq_id'):
                    df_user[i]=0
            df_user=df_user[sorted(df_user.columns.tolist())]
            df_job=df_job[sorted(df_job.columns.tolist())]

            print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
            df_user,df_job=self.clean_common_profile(df_user,df_job,'Platform')        
            df_user.to_csv(output_path+"platforms_profileuser.csv")
            df_job.to_csv(output_path+"platforms_profilejob.csv")

            #Databases
            df_user=pd.read_csv(user_profile_path+"DatabaseWorkedWith.csv",index_col='Respondent')
            df_job=pd.read_csv(job_profile_path+"databases_jobprofile.csv",index_col=0) 
            df_job.index.name='uniq_id'
            print(df_user.columns)
            print(df_job.columns)
            df_user.drop('Unnamed: 0', axis=1, inplace=True)
            df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
            df_job.columns=list(map(lambda x:x.lower(),df_job.columns))

            a=list(set(df_user.columns)-(set(df_job.columns)))
            print(a)
            for i in a:
                if(i!='Respondent'):
                    df_job[i]=0
            b=list(set(df_job.columns)-set(df_user.columns))
            print(b)
            for i in b:
                if(i!='uniq_id'):
                    df_user[i]=0
            df_user=df_user[sorted(df_user.columns.tolist())]
            df_job=df_job[sorted(df_job.columns.tolist())]

            print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
            df_user,df_job=self.clean_common_profile(df_user,df_job,'Database')        
            df_user.to_csv(output_path+"databases_profileuser.csv")
            df_job.to_csv(output_path+"databases_profilejob.csv")
  
    def match_profile(self,input_path,user_id,flag=0):
        #Match a given user_id with all jobs in the database
        
        #Check if user id exists
        df=pd.read_csv(input_path+"domain_userprofile.csv",index_col='Respondent')
  
        matches=dict()
        if(flag==0):
            if(user_id in df.index):
                userdomain=df.loc[user_id,:]
               
                df=pd.read_csv(input_path+"languages_profileuser.csv",index_col='Respondent')
                userlanguages=df.loc[user_id,:]

                df=pd.read_csv(input_path+"frameworks_profileuser.csv",index_col='Respondent')
                userframeworks=df.loc[user_id,:]

                df=pd.read_csv(input_path+"platforms_profileuser.csv",index_col='Respondent')
                userplatforms=df.loc[user_id,:]
                
                df=pd.read_csv(input_path+"databases_profileuser.csv",index_col='Respondent')
                userdatabases=df.loc[user_id,:]

                userdomain=np.asarray(userdomain.fillna(0))
                userlanguages=np.asarray(userlanguages.fillna(0))
                userframeworks=np.asarray(userframeworks.fillna(0))
                userplatforms=np.asarray(userplatforms.fillna(0))
                userdatabases=np.asarray(userdatabases.fillna(0))
                
            else:
                print("error! user id not in Dataset")
            #If it doesn't,take user profile as input
        else:

            print("New user!Enter details..")
            name=input("Enter full name")
            skills=input("Enter skills(comma separated). These are programming languages, frameworks,platforms or databases you have experience with").split(",")
            domains=''
            flag=1
            while(1):
                print("Enter domain(s) of interest separated by commas(Names are case sensitive):")
                domains=input().split(",")
                domains
                for domain in domains:
                    if(domain not in df.columns):
                        flag=0
                        break
                if(flag==1):
                    break
                else:
                    print("Please enter valid domain")
           
            skills=list(map(lambda x:x.lower(),skills))                

            userdomain=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for domain in domains:
                dictionary[domain]=1.0
            userdomain=userdomain.append(dictionary,ignore_index=True)


            df=pd.read_csv(input_path+"languages_profileuser.csv",index_col='Respondent')
            userlanguages=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userlanguages=userlanguages.append(dictionary,ignore_index=True)
            df=pd.read_csv(input_path+"frameworks_profileuser.csv",index_col='Respondent')
            userframeworks=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userframeworks=userframeworks.append(dictionary,ignore_index=True)

            df=pd.read_csv(input_path+"platforms_profileuser.csv",index_col='Respondent')
            userplatforms=pd.DataFrame(columns=df.columns)                
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userplatforms=userplatforms.append(dictionary,ignore_index=True)

            df=pd.read_csv(input_path+"databases_profileuser.csv",index_col='Respondent')
            userdatabases=pd.DataFrame(columns=df.columns)               
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userdatabases=userdatabases.append(dictionary,ignore_index=True)
            #print(userdomain)
            userdomain=np.asarray(userdomain.iloc[0,:].fillna(0))
            userlanguages=np.asarray(userlanguages.iloc[0,:].fillna(0))
            userframeworks=np.asarray(userframeworks.iloc[0,:].fillna(0))
            userplatforms=np.asarray(userplatforms.iloc[0,:].fillna(0))
            userdatabases=np.asarray(userdatabases.iloc[0,:].fillna(0))
                
        jobdomain=pd.read_csv(input_path+"domain_jobprofile.csv",index_col='uniq_id')
        joblanguages=pd.read_csv(input_path+'languages_profilejob.csv',index_col='uniq_id')
        jobframeworks=pd.read_csv(input_path+'frameworks_profilejob.csv',index_col='uniq_id')
        jobplatforms=pd.read_csv(input_path+'platforms_profilejob.csv',index_col='uniq_id')
        jobdatabases=pd.read_csv(input_path+'databases_profilejob.csv',index_col='uniq_id')
        #print(len(jobdomain.index),len(joblanguages.index))
        for i in jobdomain.index:
            #print(i)
            try:
                domain=jobdomain.loc[i,:].fillna(0)
                language=joblanguages.loc[i,:].fillna(0)
                framework=jobframeworks.loc[i,:].fillna(0)
                platform=jobplatforms.loc[i,:].fillna(0)
                database=jobdatabases.loc[i,:].fillna(0)
                job_id=str(i)
                domain=np.asarray(domain)
                language=np.asarray(language)
                framework=np.asarray(framework)
                platform=np.asarray(platform)
                database=np.asarray(database)
                #print(len(domain),len(userdomain))
                score=(0.7*cosine_similarity(domain,userdomain))+(0.3*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
                matches[job_id]=score
                
                #Initializing job profiles for later access
                self.job_domain=domain
                self.job_language=language
                self.job_framework=framework
                self.job_platform=platform
                self.job_database=database
            
                self.user_domain=userdomain
                self.user_language=userlanguages
                self.user_framework=userframeworks
                self.user_platform=userplatforms
                self.user_database=userdatabases
            except KeyError:
                pass
        matches=sorted(matches.items(),key=lambda x:x[1],reverse=True)
        print(matches[0])
       # print(type(matches))
        maxscore=matches[0][1]
       # print(maxscore)
        threshval=0.3
        threshscore=threshval*maxscore
        print("Threshold Value = "+str(threshval))
        print("Threshold Score = "+str(threshscore))
        truepositive={v for i,v in matches if v>threshscore and v>0.0}
        falsepositive={v for i,v in matches if v<threshscore and v>0.0}
        truenegative={v for i,v in matches if v==0.0}
        tp=len(truepositive)
        fp=len(falsepositive)
        tn=len(truenegative)
        tot=len(matches)
        precision=tp/(tp+fp)
        f1score=2*(precision)/(precision+1)
        print("\nPrecision = "+str(precision))
        print("\nF1 Score = "+str(f1score))
        recommendations=matches[:10]
        #print("recommendations are")
        #print(recommendations)
        rows=pd.DataFrame(columns=self.df2.columns)
        count=0
        for i in recommendations:
            row=self.df2[self.df2['uniq_id']==i[0]]
            #rows[count]=np.asarray(row.values.T.tolist()[0])
            rows=rows.append(row.iloc[0])
            count=count+1
            #print(row)
        return rows
            

In [ ]:
obj=job_postings("./data/dice_com-job_us_sample.csv")
final_cat=obj.categorize_jobs()
final_cat.to_csv("./data/preprocessed_df.csv")

    

418ff92580b270ef4e7c14f0ddfc36b4
8aec88cba08d53da65ab99cf20f6f9d9
46baa1f69ac07779274bcd90b85d9a72
[('Sales professional', 0.6358576370120061), ('Enterprise application', 0.6347311970546246), ('Front End', 0.6323985234669988), ('System Administrator', 0.610225133212027)]
3941b2f206ae0f900c4fba4ac0b18719
45efa1f6bc65acc32bbbb953a1ed13b7
e0ac9d926dda5e95162ef05adea7318c
e7e326053c586bd94e59f1fd74de4a1b
[('Back End', 0.6849856658218393)]
b0dadecf4c3c2beecb9c773ca11ecda4
28f5e0c1cc3314813e674f0c32b04d1b
95c9127e2770172f454f3b83981eaa88
[('QA/Test Developer', 0.4509044512646364)]
5e0ff38f5eaf44726f4e3d1dd257a244
e4a1ff1b6c0fda5f345e57cf1acb40dd
d0c81a2e3e5d666f3d730f1048c49132
[('Back End', 0.6526891682122978), ('Product Manager', 0.6289951734430823), ('Enterprise application', 0.6162577885084332), ('Front End', 0.6138820578802257)]
51279d060da242e3baea98b26ddd641e
48b7ca0c2f6191fb48e7ecf19fbd3322
9e5704d08bc07ddb6df9ef98b223b036
e4dceaaaaae37ca6eabb04a7d42498b4
[('Information Security', 0.

<ipython-input-51-5556c59bfc92>:39: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  match_cat_title[category]=job_title.similarity(word)


c3f2f888a9669afa04229944cfb1b0ba
f9c970d4b617a2a3304b6d0d6a20b514
[('Front End', 0.5813015433145994), ('System Administrator', 0.5789558518860449), ('Product Manager', 0.578181465885695), ('Network Engineer', 0.5707401259004595), ('Business analyst', 0.570424827073887)]
dbe60bd915c0f240963178b2c42d14c7
[('Back End', 0.6492260209864198), ('Enterprise application', 0.6154370606179492), ('Front End', 0.6088392609317187), ('System Administrator', 0.6042706299655901)]
1e5ca7acf2df0830333437c8e1cccde4
[('QA/Test Developer', 0.447897057345551)]
3cc6c270f68b5008d66df9a3250f7236
[('Business analyst', 0.49106682456414363), ('Game developer', 0.48550455686452076), ('Data Scientist', 0.4825013684142991), ('Mobile Developer', 0.47134535376923065), ('Software Developer/Java Developer', 0.45941940027445755)]
dc5cb301e40afe9509a1840b29764794
[('Database Administrator(DBA)', 0.5425632080321314), ('Sales professional', 0.5339691280695052), ('Full stack', 0.5143109631114887), ('Enterprise application', 0

In [22]:
obj=job_postings("./data/dice_com-job_us_sample.csv")
extracted_skills=obj.clean_skills()
extracted_skills_df=obj.extract_skills(extracted_skills)
print(extracted_skills_df)            


[('418ff92580b270ef4e7c14f0ddfc36b4', ['', '', '', '', '', 'see below']), ('8aec88cba08d53da65ab99cf20f6f9d9', ['', '', '', 'linux,unix', '', 'incident response,network monitoring,systems administration,security accessment']), ('46baa1f69ac07779274bcd90b85d9a72', ['', '', '', '', '', 'reporting,business inteligence,enterprise solutions architecture,reports']), ('3941b2f206ae0f900c4fba4ac0b18719', ['', '', '', '', '', 'please see job description']), ('45efa1f6bc65acc32bbbb953a1ed13b7', ['', '', '', 'linux', '', 'configuration management,management,vmware,process engineering,developer']), ('e0ac9d926dda5e95162ef05adea7318c', ['', '', '', '', '', 'ar,ap,fico,asset management,haha']), ('e7e326053c586bd94e59f1fd74de4a1b', ['', '', '', '', '', 'networking,http,security,video,dns,network engineer,vpn,wireless,cisco']), ('b0dadecf4c3c2beecb9c773ca11ecda4', ['.net,c#', '', '', '', 'azure,aws', 'http,restful web services,mvc,oop']), ('28f5e0c1cc3314813e674f0c32b04d1b', ['javascript,c++', '', '',

In [26]:
extracted_skills_df.head(13)

,Language,Framework,Database,OS,Platform,Others
418ff92580b270ef4e7c14f0ddfc36b4,",groovy,java",,,,,see below
8aec88cba08d53da65ab99cf20f6f9d9,,,,"linux,unix",,"incident response,network monitoring,systems a..."
46baa1f69ac07779274bcd90b85d9a72,,,,,,"reporting,business inteligence,enterprise solu..."
3941b2f206ae0f900c4fba4ac0b18719,",java,c++",,",oracle",",aix",,please see job description
45efa1f6bc65acc32bbbb953a1ed13b7,",puppet",,,linux,,"configuration management,management,vmware,pro..."
e0ac9d926dda5e95162ef05adea7318c,,,,,,"ar,ap,fico,asset management,haha"
e7e326053c586bd94e59f1fd74de4a1b,,,,,,"networking,http,security,video,dns,network eng..."
b0dadecf4c3c2beecb9c773ca11ecda4,".net,c#",,,,"azure,aws","http,restful web services,mvc,oop"
28f5e0c1cc3314813e674f0c32b04d1b,"javascript,c++",,,,,"user interface,developer,development"
95c9127e2770172f454f3b83981eaa88,",java,javascript",,,,",ios",(see job description)


In [46]:

obj=job_postings("./data/dice_com-job_us_sample.csv")
domain_df=pd.read_csv("./data/preprocessed_df.csv")
obj.create_job_profile(extracted_skills_df,domain_df)


['.net', 'abap', 'abc', 'actionscript', 'ada', 'ajax', 'apex', 'apl', 'applescript', 'arduino', 'asp', 'assembly', 'atlas', 'automator', 'bash', 'bc', 'bourne shell', 'bro', 'c', 'c shell', 'c#', 'c++', 'caml', 'ceylon', 'cfml', 'cg', 'clarion', 'clojure', 'cobol', 'cobra', 'coffeescript', 'coldfusion', 'css', 'ct', 'd', 'dart', 'dcl', 'e', 'ec', 'ecl', 'ecmascript', 'egl', 'elixir', 'erlang', 'f#', 'falcon', 'forth', 'fortran', 'fortress', 'go', 'gosu', 'groovy', 'haskell', 'html', 'idl', 'inform', 'informix-4gl', 'io', 'java', 'javascript', 'jscript', 'julia', 'korn shell', 'kotlin', 'labview', 'ladder logic', 'lingo', 'lisp', 'logo', 'lotusscript', 'lpc', 'lua', 'lustre', 'magic', 'mathematica', 'matlab', 'mercury', 'ml', 'moo', 'mumps', 'natural', 'objective-c', 'occam', 'opa', 'opencl', 'perl', 'php', 'pl/sql', 'powerscript', 'powershell', 'prolog', 'puppet', 'python', 'q', 'r', 'rexx', 'ruby', 'ruby on rails', 'rust', 's', 'sas', 'scala', 'scilab', 'sed', 'self', 'shell', 'signal

In [51]:
obj=job_postings("./data/dice_com-job_us_sample.csv")
obj.create_common_profile("./data/job_profile/","./data/user/","./data/")

Read from file
Index(['418ff92580b270ef4e7c14f0ddfc36b4', '8aec88cba08d53da65ab99cf20f6f9d9',
       '46baa1f69ac07779274bcd90b85d9a72', '3941b2f206ae0f900c4fba4ac0b18719',
       '45efa1f6bc65acc32bbbb953a1ed13b7', 'e0ac9d926dda5e95162ef05adea7318c',
       'e7e326053c586bd94e59f1fd74de4a1b', 'b0dadecf4c3c2beecb9c773ca11ecda4',
       '28f5e0c1cc3314813e674f0c32b04d1b', '95c9127e2770172f454f3b83981eaa88',
       ...
       'ba32786b50e8b4d468cb9d813066fea5', '541b0f16ecd86fdbd7ee2b04523dc65e',
       'ee46660dd5318edb27b229f51341fcf8', '4b1726afa64b7c9dc640c7862ae64ecc',
       '1acc1a7a845f0b9c233ad42941f0462d', '86e27ce6b7e631e55d69d142c7d43df2',
       '4287c7ee3317ccf1edd76e238cf8e584', 'd7512f0181d69f83f96db38cd77a4d08',
       'ec375268b494b3bcbed1635d64226112', '9a4e8c27f74af4c0d2f6efbd420a8a91'],
      dtype='object', name='Unnamed: 0', length=22000)
index 2in domain
Index(['418ff92580b270ef4e7c14f0ddfc36b4', '8aec88cba08d53da65ab99cf20f6f9d9',
       '46baa1f69ac07779274bcd90

In [7]:
obj=job_postings("./data/dice_com-job_us_sample.csv")
rows=obj.match_profile("./data/",5)
rows

('3c16025689421ae36b3400191f3eeb63', 1.282842712474619)
Threshold Value = 0.3
Threshold Score = 0.38485281374238567

Precision = 0.11575031004547334

F1 Score = 0.2074842534271953


,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
5609,https://www.dice.com/jobs/detail/Full-Stack-SD...,Wicresoft NA,"Full Time, Contract Corp-To-Corp, Contract W2,...",Please send your resume to Crystall@wicresoft....,Dice Id : RTX15f147,"Redmond, WA",Full Stack SDE,2 weeks ago,Telecommuting not available|Travel not required,NaN,C# AND JavaScript AND Server,3c16025689421ae36b3400191f3eeb63
15885,https://www.dice.com/jobs/detail/Full-Stack-De...,Hub International,Full Time,About Hub InternationalHUB International Limit...,Dice Id : RTX174b51,"Chicago, IL",Full Stack Developer,2 weeks ago,Telecommuting not available|Travel not required,www.dice.com,"HTML5, jQuery, CSS, AngularJS, JavaScript, Saa...",4c16b6641f2aa3e68aed632876c0014a
4143,https://www.dice.com/jobs/detail/5-Full-Stack-...,Vaco - San Francisco,"Contract Corp-To-Corp, Contract W2, 24 - 36 mo...",Vaco is seeking five (5) Full Stack Developers...,Dice Id : 10115369,"Sunnyvale, CA",5 Full Stack Engineers - Long Term Engagements,16 minutes ago,Telecommuting not available|Travel not required,NaN,"Node.js, Bootstrap, Skeleton, jQuery, REST API...",fee541ff1e2c9acdbd223398e6f70050
2408,https://www.dice.com/jobs/detail/Full-Stack-De...,AETEA Information Technology Inc,"C2H Corp-To-Corp, C2H Independent, C2H W2","Aetea has been asked by their valued client, a...",Dice Id : cxaetea,"Wayne, PA",Full Stack Developer,1 week ago,Telecommuting not available|Travel not required,NaN,"Full Stack Developer, C#, MVC, SQL Server, JQu...",cf4a71d87a3b46cf00d35317b76f3767
5871,https://www.dice.com/jobs/detail/Full-Stack-En...,Dew Softech Inc,"Contract Corp-To-Corp, C2H Corp-To-Corp, 6 Con...",Position: Full Stack EngineerLocation...,Dice Id : 10243845,"Houston,, TX",Full Stack Engineer,5 hours ago,Telecommuting not available|Travel not required,NaN,"2+ years experience with building Python, Node...",2e88654d255563f4d86baf821f92a8ac
21083,https://www.dice.com/jobs/detail/Full-Stack-De...,Bay Area Techworkers,"Contract Corp-To-Corp, Contract Independent, C...",***Local candidates only***F2F required***Plea...,Dice Id : bati,"San Ramon, CA",Full Stack Developer,1 week ago,Telecommuting not available|Travel not required,NaN,"Agile, Java, Spring, Hibernate, RESTful, WebSe...",7e71d56a0a0f62d3e4e062b84131da29
4431,https://www.dice.com/jobs/detail/Full-stack-En...,Myriad Consulting Inc,Contract W2,Job Title: Full Stack EngineerStart Date: ASAP...,Dice Id : 90940280,"Redmond, WA",Full stack Engineer,1 day ago,Telecommuting not available|Travel not required,NaN,"Full stack, Azure, Web, Cosmos, hadoop",5dd7bf5b767ca37a492e2a2d673befd0
5779,https://www.dice.com/jobs/detail/Full-Stack-En...,Techgene Solutions LLC,"Full Time, Contract Corp-To-Corp, Contract Ind...",Position – Full Stack EngineerLocation – Houst...,Dice Id : 10109301,"Houston, TX",Full Stack Engineer,3 days ago,Telecommuting not available|Travel not required,NaN,"C#, ASP.NET, Python, AWS, Google Cloud, Azure",6611fbe1b740c2c29d4365ff9d60eefa
19359,https://www.dice.com/jobs/detail/Java-Develope...,"Sterling 5, Inc.","Contract Corp-To-Corp, Contract Independent, C...",Role: Java Developer- Full stack Location: San...,Dice Id : 10431828,"San Ramon, CA",Java Developer- Full stack,18 hours ago,Telecommuting not available|Travel not required,NaN,"Spring MVC, JPA, REST, Jackson, SOA, JPA",deb8ac91cb527574b00ae99f49cf1b3d
21976,https://www.dice.com/jobs/detail/Senior-Full-S...,Net2Source Inc.,"Contract Corp-To-Corp, Contract Independent, C...","Net2Source, Inc. is one of the fastest growing...",Dice Id : 10271304,"San Ramon, CA",Senior Full Stack Developer,3 weeks ago,Telecommuting not available|Travel not required,NaN,Null,56b2dd205154a1cb2eae101efcd14567


In [70]:
obj=job_postings("./data/dice_com-job_us_sample.csv")
rows=obj.match_profile("./data/",10)
rows

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
780,https://www.dice.com/jobs/detail/Front-End-Sof...,Thorndale Partners LLC,Full Time,FRONT END SOFTWARE ENGINEER DETAILSTitle: Fron...,Dice Id : 10523569,"Boston, MA",Front End Software Engineer,6 days ago,Telecommuting not available|Travel not required,www.dice.com,"Javascript, html, css, jquery",9055f964bd964b4c4740319f0b558a16
16752,https://www.dice.com/jobs/detail/Front-End-Sof...,Thorndale Partners LLC,Full Time,FRONT END SOFTWARE ENGINEER DETAILSTitle: Fron...,Dice Id : 10523569,"Boston, MA",Front End Software Engineer,3 days ago,Telecommuting not available|Travel not required,NaN,"Javascript, html, css, jquery",5d61a739ec126e005aaae39acc3b5cbc
1830,https://www.dice.com/jobs/detail/Front-End-Dev...,Austin Fraser USA,Full Time,What do we do?We develop state of the art soft...,Dice Id : 10347650,"Dallas, TX",Front End Developer,2 hours ago,Telecommuting not available|Travel not required,www.dice.com,html AND css AND less AND bootstrap AND javasc...,d6539633a60df2ecf0b797fbac5e3a94
21075,https://www.dice.com/jobs/detail/Front%2526%25...,Payette Group,Full Time,2-4 years ExperienceJavaScript & CSS 3+ years;...,Dice Id : payette,"San Francisco, CA",Front-end Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"JavaScript, HTML, CSS",4f64f86acb4a92f0fade4103d664d204
16700,https://www.dice.com/jobs/detail/UI-Software-E...,NCR Corporation,Full Time,Description NCR Corporation (NYSE: NCR) is the...,Dice Id : NCRGA,"Orlando, FL",UI Software Enginer,3 days ago,Telecommuting not available|Travel not required,NaN,"Software Development, Javascript, coffereescri...",82c3bc53960d10d3b764ad3bc0cd87a2
21688,https://www.dice.com/jobs/detail/Sr.-Front%252...,"One-To-One Staffing, Inc.","Full Time, full-time","As a member of this agile team, you will be re...",Dice Id : MISKEY,"San Francisco, CA",Sr. Front-end Developer,3 weeks ago,Telecommuting not available|Travel not required,NaN,"Javascript, various JS frameworks, CSS, Ruby a...",6667721f289bf1af1c5eb33ff52cd95e
15026,https://www.dice.com/jobs/detail/Front-End-Dev...,Kalo Solutions,Full Time,Our client is looking for a Front End Develope...,Dice Id : 90688978,"Chicago, IL",Front End Developer,5 days ago,Telecommuting not available|Travel not required,NaN,"JavaScript, CMS, HTML, CSS, C#, Downtown, Fron...",bd3b8568d0dbf13195bb0beb9fd48c29
18852,https://www.dice.com/jobs/detail/Sr.-Front%252...,OnX Enterprise Solutions,"Contract Independent, Contract W2, 6 months",Client is making changes and enhancements to c...,Dice Id : 10117905d,"New York, NY",Sr. Front-End Developer,4 days ago,Telecommuting not available|Travel not required,NaN,JavaScript and HTML and Angular and automated ...,8d82d62f36953589f4764c881f929033
5009,https://www.dice.com/jobs/detail/.net-front-en...,Reliable Software Resources,Contract W2,"Hi, This is Pranay from Reliable Software Reso...",Dice Id : 10122703,"Redmond, WA",.net front end developer,1 week ago,Telecommuting not available|Travel not required,NaN,.Net Front-End Software Development Engineer,1f72954dc331b3e6b2dfc5f54a37632a
6790,https://www.dice.com/jobs/detail/Front%2526%25...,"firstPRO, Inc.",Full Time,"Fast-growing, stable company has an immediate ...",Dice Id : FIRSTPRO,"Alpharetta, GA",Front-End Developer / Ember.js,3 days ago,Telecommuting not available|Travel not required,NaN,"javascript, ember, CSS",087413bbc8d27c5ccc40550c8fd51089


In [72]:

rows=obj.match_profile("./data/",5,1)
rows

New user!Enter details..
Enter full nameSara
Enter skills(comma separated). These are programming languages, frameworks,platforms or databases you have experience withjava,python,c++,android,oracle,mysql,linux,react,Spring,struts
Enter domain(s) of interest separated by commas(Names are case sensitive):
Software Developer/Java Developer,Web Developer


,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
11030,https://www.dice.com/jobs/detail/Senior-Java-D...,Connexin Software,Full Time,Connexin developers build software that connec...,Dice Id : RTX170079,"Horsham, PA",Senior Java Developer,4 weeks ago,Telecommuting available|Travel not required,NaN,"Java, ORM Frameworks Hibernate, JOOQ, MySQL, S...",a0d8cce1effacc756a8e4b5ed617c913
6743,https://www.dice.com/jobs/detail/Senior-Java-D...,Applied Resource Group,Full Time,Java Software Engineer:Unable to sponsor H1B V...,Dice Id : 10447354,"Alpharetta, GA",Senior Java Developer,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Java, C++, DB2 databases, Oracle, Linux, UNIX ...",7ee6cb797e1915e4b4441a0777088094
1285,https://www.dice.com/jobs/detail/Locals-ONLY-%...,"Nueva Solutions, Inc.","Contract Corp-To-Corp, Contract Independent, C...","LocationWindsor, CT - In Person interview mand...",Dice Id : 10359170,"Windsor, CT",Locals ONLY / In-person Interview: Java Develo...,6 days ago,Telecommuting not available|Travel not required,www.dice.com,"Java,Angular JS,JDBC,Junit,Jquery,Ajax ,Hibernate",308dc2b263744114cccdd0ea382f49e4
12249,https://www.dice.com/jobs/detail/Senior-J2EE%2...,Spring Lake Consulting,"Full Time, Contract Independent, Contract W2, ...",***** H1B Transfers Available**We are looking ...,Dice Id : 10112272,"Irving, TX",Senior J2EE/Java Developer,2 days ago,Telecommuting not available|Travel not required,NaN,"Java/J2EE, Spring, Struts, ESB, Design Pattern...",ddb2f8eb3966138d67028f5d5cce1518
745,https://www.dice.com/jobs/detail/Java-Develope...,Resourcesys,"Full Time, Long Term",Job Title/Role : Java DeveloperLocation : Fram...,Dice Id : 10170376,"Framingham, MA",Java Developer (Full Time),2 weeks ago,Telecommuting not available|Travel not required,www.dice.com,"java, Spring, Struts, Hibernate, Webservices, ...",d26a38497c95d812b7bce31524ab6f43
8525,https://www.dice.com/jobs/detail/5-Senior-Java...,AMS Staffing Inc.,"Full Time, Fulltime",Please send resume in Word format if you are i...,Dice Id : 10192616,"Plano, TX",5 Senior Java Developers,1 day ago,Telecommuting not available|Travel not required,NaN,"Bachelor's Degree, Java, J2EE, Spring, Hiberna...",231b9575714a333ba75f59574fc588dd
9923,https://www.dice.com/jobs/detail/Java-Mid%2526...,LiquidHub,"Full Time, Contract W2, Open","LiquidHub is a digital integrator, a next gene...",Dice Id : 10111907,"Wayne, PA",Java Mid-tier Developers,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Java, J2EE, MySQL, Spring",e5d9abaf3f5e355cdcb8e12fb83bc253
20319,https://www.dice.com/jobs/detail/Sr-Software-E...,Group A LLC,"C2H Corp-To-Corp, C2H Independent, C2H W2, 6 m...",*PLEASE APPLY WITH A WORD COPY OF YOUR RESUME...,Dice Id : 10364869,"San Francisco, CA",Sr Software Engineer-Java,2 days ago,Telecommuting not available|Travel not required,NaN,"JAVE/J2EE, CUSTOMER INTERACTION/SUPPORT, DEVEL...",19c60022adf0d3f9ec5c853057b88c1a
783,https://www.dice.com/jobs/detail/Java-Develope...,Cypress Group,"Full Time, Full-Time",Our client believes that when people come toge...,Dice Id : 10106564,"Lexington, MA",Java Developer,2 days ago,Telecommuting not available|Travel not required,www.dice.com,"Java, Spring, Web Services",098008a35f4104c3df1130bf097db50c
6869,https://www.dice.com/jobs/detail/Sr.-Java-Deve...,APN Consulting Inc,"Contract Corp-To-Corp, Contract Independent, C...",DIRECT CLIENT REQUIREMENTTitle: Sr. Java Devel...,Dice Id : 10123488,"Alpharetta, GA",Sr. Java Developer,1 week ago,Telecommuting not available|Travel not required,NaN,"Java , j2ee, Spring, RESTful services, web ser...",08359d66b58500e4019ef0b9cd5f05c8


In [73]:
rows=obj.match_profile("./data/",5,1)
rows

New user!Enter details..
Enter full nameritka
Enter skills(comma separated). These are programming languages, frameworks,platforms or databases you have experience withjava,python,spring,android,oracle,mysql,struts,spring
Enter domain(s) of interest separated by commas(Names are case sensitive):
Software Developer/Java Developer


,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
11030,https://www.dice.com/jobs/detail/Senior-Java-D...,Connexin Software,Full Time,Connexin developers build software that connec...,Dice Id : RTX170079,"Horsham, PA",Senior Java Developer,4 weeks ago,Telecommuting available|Travel not required,NaN,"Java, ORM Frameworks Hibernate, JOOQ, MySQL, S...",a0d8cce1effacc756a8e4b5ed617c913
12249,https://www.dice.com/jobs/detail/Senior-J2EE%2...,Spring Lake Consulting,"Full Time, Contract Independent, Contract W2, ...",***** H1B Transfers Available**We are looking ...,Dice Id : 10112272,"Irving, TX",Senior J2EE/Java Developer,2 days ago,Telecommuting not available|Travel not required,NaN,"Java/J2EE, Spring, Struts, ESB, Design Pattern...",ddb2f8eb3966138d67028f5d5cce1518
745,https://www.dice.com/jobs/detail/Java-Develope...,Resourcesys,"Full Time, Long Term",Job Title/Role : Java DeveloperLocation : Fram...,Dice Id : 10170376,"Framingham, MA",Java Developer (Full Time),2 weeks ago,Telecommuting not available|Travel not required,www.dice.com,"java, Spring, Struts, Hibernate, Webservices, ...",d26a38497c95d812b7bce31524ab6f43
8525,https://www.dice.com/jobs/detail/5-Senior-Java...,AMS Staffing Inc.,"Full Time, Fulltime",Please send resume in Word format if you are i...,Dice Id : 10192616,"Plano, TX",5 Senior Java Developers,1 day ago,Telecommuting not available|Travel not required,NaN,"Bachelor's Degree, Java, J2EE, Spring, Hiberna...",231b9575714a333ba75f59574fc588dd
9923,https://www.dice.com/jobs/detail/Java-Mid%2526...,LiquidHub,"Full Time, Contract W2, Open","LiquidHub is a digital integrator, a next gene...",Dice Id : 10111907,"Wayne, PA",Java Mid-tier Developers,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Java, J2EE, MySQL, Spring",e5d9abaf3f5e355cdcb8e12fb83bc253
20319,https://www.dice.com/jobs/detail/Sr-Software-E...,Group A LLC,"C2H Corp-To-Corp, C2H Independent, C2H W2, 6 m...",*PLEASE APPLY WITH A WORD COPY OF YOUR RESUME...,Dice Id : 10364869,"San Francisco, CA",Sr Software Engineer-Java,2 days ago,Telecommuting not available|Travel not required,NaN,"JAVE/J2EE, CUSTOMER INTERACTION/SUPPORT, DEVEL...",19c60022adf0d3f9ec5c853057b88c1a
783,https://www.dice.com/jobs/detail/Java-Develope...,Cypress Group,"Full Time, Full-Time",Our client believes that when people come toge...,Dice Id : 10106564,"Lexington, MA",Java Developer,2 days ago,Telecommuting not available|Travel not required,www.dice.com,"Java, Spring, Web Services",098008a35f4104c3df1130bf097db50c
6869,https://www.dice.com/jobs/detail/Sr.-Java-Deve...,APN Consulting Inc,"Contract Corp-To-Corp, Contract Independent, C...",DIRECT CLIENT REQUIREMENTTitle: Sr. Java Devel...,Dice Id : 10123488,"Alpharetta, GA",Sr. Java Developer,1 week ago,Telecommuting not available|Travel not required,NaN,"Java , j2ee, Spring, RESTful services, web ser...",08359d66b58500e4019ef0b9cd5f05c8
10132,https://www.dice.com/jobs/detail/Senior-Softwa...,"NCS Technologies, Inc.","Contract Corp-To-Corp, Contract Independent, C...","Senior Software Engineer : Princeton, NJ Cont...",Dice Id : ncstinj,"Princeton, NJ",Senior Software Engineer/CMS,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Senior Software Engineer/CMS : Princeton, NJ",901a7b1f7de02f93668e1ffbab5359c3
14215,https://www.dice.com/jobs/detail/Java-Webservi...,"IRIS Software, Inc.","Contract Independent, Contract W2, C2H Indepen...","Iris's client, one of the world's prominent fi...",Dice Id : irissoft,"Pennington, NJ",Java Webservices Developer / Middleware Develo...,2 days ago,Telecommuting not available|Travel not required,NaN,"Java, J2ee, Spring, Hibernate, Webservices, Re...",75c86c8613745841aef2abf15d36ed62


In [1]:
import flask
import difflib

app = flask.Flask(__name__, template_folder='templates')


In [ ]:
def getecommendations(title):
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    tit = df2['title'].iloc[movie_indices]
    dat = df2['release_date'].iloc[movie_indices]
    return_df = pd.DataFrame(columns=['Title','Year'])
    return_df['Title'] = tit
    return_df['Year'] = dat
    return return_df


In [9]:
def get_recommendations(skills,domains):
        df=pd.read_csv("./data/domain_userprofile.csv",index_col='Respondent')
        matches=dict()
        skills=list(map(lambda x:x.lower(),skills))                
        input_path="./data/"
        userdomain=pd.DataFrame(columns=df.columns)
        dictionary=dict()
            
        for domain in domains:
            dictionary[domain]=1.0
        userdomain=userdomain.append(dictionary,ignore_index=True)


        df=pd.read_csv(input_path+"languages_profileuser.csv",index_col='Respondent')
        userlanguages=pd.DataFrame(columns=df.columns)
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userlanguages=userlanguages.append(dictionary,ignore_index=True)
        df=pd.read_csv(input_path+"frameworks_profileuser.csv",index_col='Respondent')
        userframeworks=pd.DataFrame(columns=df.columns)
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userframeworks=userframeworks.append(dictionary,ignore_index=True)
        df=pd.read_csv(input_path+"platforms_profileuser.csv",index_col='Respondent')
        userplatforms=pd.DataFrame(columns=df.columns)                
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userplatforms=userplatforms.append(dictionary,ignore_index=True)

        df=pd.read_csv(input_path+"databases_profileuser.csv",index_col='Respondent')
        userdatabases=pd.DataFrame(columns=df.columns)               
        dictionary=dict()
        for skill in skills:
            if(skill in df.columns):
                dictionary[skill]=1.0
        userdatabases=userdatabases.append(dictionary,ignore_index=True)
            #print(userdomain)
        userdomain=np.asarray(userdomain.iloc[0,:].fillna(0))
        userlanguages=np.asarray(userlanguages.iloc[0,:].fillna(0))
        userframeworks=np.asarray(userframeworks.iloc[0,:].fillna(0))
        userplatforms=np.asarray(userplatforms.iloc[0,:].fillna(0))
        userdatabases=np.asarray(userdatabases.iloc[0,:].fillna(0))
                
        jobdomain=pd.read_csv(input_path+"domain_jobprofile.csv",index_col='uniq_id')
        joblanguages=pd.read_csv(input_path+'languages_profilejob.csv',index_col='uniq_id')
        jobframeworks=pd.read_csv(input_path+'frameworks_profilejob.csv',index_col='uniq_id')
        jobplatforms=pd.read_csv(input_path+'platforms_profilejob.csv',index_col='uniq_id')
        jobdatabases=pd.read_csv(input_path+'databases_profilejob.csv',index_col='uniq_id')
        #print(len(jobdomain.index),len(joblanguages.index))
        for i in jobdomain.index:
            #print(i)
            try:
                domain=jobdomain.loc[i,:].fillna(0)
                language=joblanguages.loc[i,:].fillna(0)
                framework=jobframeworks.loc[i,:].fillna(0)
                platform=jobplatforms.loc[i,:].fillna(0)
                database=jobdatabases.loc[i,:].fillna(0)
                job_id=str(i)
                domain=np.asarray(domain)
                language=np.asarray(language)
                framework=np.asarray(framework)
                platform=np.asarray(platform)
                database=np.asarray(database)
                #print(len(domain),len(userdomain))
                score=(0.7*cosine_similarity(domain,userdomain))+(0.3*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
                matches[job_id]=score
                score=(0.7*cosine_similarity(domain,userdomain))+(0.3*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
                #Initializing job profiles for later access
                
            except KeyError:
                pass
        matches=sorted(matches.items(),key=lambda x:x[1],reverse=True)
        recommendations=matches[:10]
        df2=pd.read_csv("./data/dice_com-job_us_sample.csv")
        rows=pd.DataFrame(columns=df2.columns)
        count=0
        for i in recommendations:
            row=df2[df2['uniq_id']==i[0]]
            #rows[count]=np.asarray(row.values.T.tolist()[0])
            rows=rows.append(row.iloc[0])
            count=count+1
            #print(row)
        return rows
            

In [26]:
@app.route('/',endpoint='uonii',methods=['GET','POST'])

def uonii():
    if flask.request.method == 'GET':
        return(flask.render_template('index_rec.html'))
            
    if flask.request.method == 'POST':
    
        #.list()
        d=flask.request.form['domain']
        s=flask.request.form['skills']
        
        df=pd.read_csv("./data/domain_userprofile.csv",index_col='Respondent')
        #        check = difflib.get_close_matches(m_name,all_titles,cutout=0.50,n=1)
        domains=d.split(",")
        
        
        r=get_recommendations(s,domains)
        return render_template('positive.html',  tables=[r.to_html(classes='data')], titles=r.columns.values)
if __name__ == '__main__':
    app.run(host='localhost',port='5001',debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001/ (Press CTRL+C to quit)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/May/2021 02:07:10] "GET / HTTP/1.1" 200 -
